In [1]:
%load_ext Cython

import pandas as pd
import numpy as np
import datetime
import statistics
import pickle
import random
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from sklearn.model_selection import train_test_split

In [2]:
from recommender_algo.editable_svd import EditableSVD

In [3]:
def top_n(user_id, movies_map, user_rated_items, algo):
    top_n = []
    for i in movies_map:
        # Filter out rated movies
        if i not in user_rated_items:
            prediction = algo.predict(user_id, i)
            top_n.append((prediction.iid, prediction.est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    top_n.sort(key=lambda x: x[1], reverse=True)
    return top_n[:30]

In [4]:
ratings_df = pd.read_csv("../data/ml_100k/ratings.csv", dtype={
    'userId': np.int32,
    'movieId': np.int32,
    'rating': np.float32,
    'timestamp': np.int32,
})
trainset, testset = train_test_split(ratings_df, test_size=.25, random_state=42)
print(trainset)
reader = Reader(rating_scale=(1, 5))

# Init list of movies
movies_df = pd.read_csv("../data/ml_100k/movies.csv", dtype={
   'movieId': int,
   'title': str,
   'genres': str,
})
movies_map = {}
for index, row in movies_df.iterrows():
    movies_map[row['movieId']] = row['title']

        userId  movieId  rating   timestamp
97717      606     2145     3.5  1171404887
100124     610    32029     3.0  1493847618
25952      180      785     2.0  1270237531
25871      178     7361     4.5  1164354934
97255      605     4993     5.0  1277175627
...        ...      ...     ...         ...
6265        42     4005     4.0   996259059
54886      364      141     4.0   869443367
76820      480     6867     4.0  1179163171
860          6      981     3.0   845556567
15795      103     6711     5.0  1431957425

[75627 rows x 4 columns]


In [5]:
user_id = 542848092
influence_precisions = []
control_precisions = []
for i in range(50):
    # Get movies
    movies_to_rate = []
    for k in range(20):
        key = random.choice(list(movies_map.keys()))
        movies_to_rate.append(key)
    user_rated_items = { key: random.randint(1, 5) for key in movies_to_rate }
    
    # Train one without films
    trainset_without = Dataset.load_from_df(trainset[['userId', 'movieId', 'rating']], reader).build_full_trainset()
    influence_algo = EditableSVD()
    influence_algo.fit(trainset_without)
    influence_algo.fit_new_user(user_id, user_rated_items)
    
    # Train one with films
    enhanced_trainset = trainset.copy()
    for (key, rating) in user_rated_items.items():
        enhanced_trainset = enhanced_trainset.append(pd.Series([user_id, key, rating, 0], 
           index=['userId', 'movieId', 'rating', 'timestamp']), ignore_index=True)
    trainset_enhanced = Dataset.load_from_df(enhanced_trainset[['userId', 'movieId', 'rating']], reader).build_full_trainset()
    print(str(len(enhanced_trainset)))
    
    baseline_algo = SVD()
    baseline_algo.fit(trainset_enhanced)
    # Train one with films (control)
    control_algo = SVD()
    control_algo.fit(trainset_enhanced)
    
    # Get top-20 recommendations for all
    baseline_top_n = top_n(user_id, movies_map, user_rated_items, baseline_algo)
    influence_top_n = top_n(user_id, movies_map, user_rated_items, influence_algo)
    control_top_n = top_n(user_id, movies_map, user_rated_items, control_algo)
    
    # Calculate precision for optimisation
    influence_precision = 0
    for (key, rating) in influence_top_n:
        for (key_base, rating_base) in baseline_top_n:
            if key == key_base:
                influence_precision += 1
    
    # Calculate precision for control
    control_precision = 0
    for (key, rating) in control_top_n:
        for (key_base, rating_base) in baseline_top_n:
            if key == key_base:
                control_precision += 1
    
    # Add these to overall ones
    print("Influence: " + str(influence_precision / 30))
    influence_precisions.append(influence_precision / 30)
    print("Control: " + str(control_precision / 30))
    control_precisions.append(control_precision / 30)
    
# Calculate mean of precision for both
print("Mean_influence: " + str(statistics.mean(influence_precisions)) 
      + ", std_influence: " + str(statistics.stdev(influence_precisions)))
print("Mean_control: " + str(statistics.mean(control_precisions)) 
      + ", std_control: " + str(statistics.stdev(control_precisions)))

75647
Influence: 0.43333333333333335
Control: 0.3
75647
Influence: 0.23333333333333334
Control: 0.23333333333333334
75647
Influence: 0.3
Control: 0.3333333333333333
75647
Influence: 0.2
Control: 0.43333333333333335
75647
Influence: 0.36666666666666664
Control: 0.16666666666666666
75647
Influence: 0.23333333333333334
Control: 0.26666666666666666
75647
Influence: 0.3
Control: 0.43333333333333335
75647
Influence: 0.26666666666666666
Control: 0.16666666666666666
75647
Influence: 0.3333333333333333
Control: 0.3333333333333333
75647
Influence: 0.4
Control: 0.3
75647
Influence: 0.26666666666666666
Control: 0.2
75647
Influence: 0.3333333333333333
Control: 0.4
75647
Influence: 0.4
Control: 0.4666666666666667
75647
Influence: 0.26666666666666666
Control: 0.2
75647
Influence: 0.26666666666666666
Control: 0.23333333333333334
75647
Influence: 0.16666666666666666
Control: 0.2
75647
Influence: 0.13333333333333333
Control: 0.26666666666666666
75647
Influence: 0.26666666666666666
Control: 0.23333333333